In [1]:
import os
import pandas as pd
import numpy as np
import random
import dendropy
import treeswift
from treeswift import read_tree_newick
#import matplotlib.pyplot as plt
import re

In [10]:
print(os.getcwd())

/Users/nora/Documents/ml_metagenomics/tol_variable_k_resuls/revisions_ONT_reads_variable_err


In [95]:
path = "results"   # change to your folder

# list only directories (not files)
dirs = [d for d in os.listdir(path) if os.path.isdir(os.path.join(path, d)) ]

In [96]:
dirs

['ONT_095_Claded_Chunked_TRUE',
 'ONT_095_Claded_Chunked',
 'ONT_090_Claded_Chunked_TRUE',
 'ONT_085_Claded_Chunked_TRUE',
 'ONT_099_Uncladed_Chunked',
 'ONT_090_Claded_Chunked',
 'ONT_085_Claded_Chunked',
 'ONT_099_Claded_Chunked',
 'ONT_099_Claded_Chunked_TRUE',
 'ONT_1_Claded_Chunked',
 'ONT_085_Uncladed_Chunked',
 'ONT_095_Uncladed_Chunked',
 'ONT_1_Uncladed_Chunked',
 'ONT_090_Uncladed_Chunked',
 'ONT_1_Claded_Chunked_TRUE']

In [97]:
my_path = "/Users/nora/Documents/ml_metagenomics"
length_folder = "ONT_read_length"

true_clades = pd.read_csv(os.path.join(my_path, "clade_targets.txt"), sep = " ", header = 0)

In [98]:
true_clades

,genome,clade
0,G001940755,13
1,G001918745,13
2,G001920395,13
3,G001918475,13
4,G000724145,13
...,...,...
10570,G001652645,9
10571,G001652685,9
10572,G900061685,9
10573,G900061725,9


In [102]:
my_big_df = pd.DataFrame()

for d in dirs:
    
    dir_path = os.path.join(path, d)
    
    if os.path.isdir(dir_path):
        # Define the two files you want to read
        file1 = os.path.join(dir_path, "classes.out")
        file2 = os.path.join(dir_path, "all.pl_err" )

        # Only proceed if both files exist
        if os.path.exists(file1) and os.path.exists(file2):
            df1 = pd.read_csv(file1, sep="\t")
            
            df2 = pd.read_csv(file2, sep=" ", header = None, names=["contigID", "empty", "count", "pl_err", "ratio"] )
            df2 = df2.drop(columns=["empty"])
            try:
                filtered_df2 = df2[df2["contigID"] != "all.pl_err"]
            except:
                filtered_df2=copy.deepcopy(df2)
            # Remove the prefix from each entry in 'contigID'
            filtered_df2.loc[:, "contigID"] = filtered_df2["contigID"].str.replace("apples_input_di_mtrx_query_", "", regex=False)
            filtered_df2.loc[:, "contigID"] = filtered_df2["contigID"].str.replace(".pl_err", "", regex=False)


        combo_df = pd.merge(filtered_df2, df1, how = "left", left_on = "contigID", right_on = "genome")
        
        # Extract everything before ".part"
        combo_df["parent_genome"] = combo_df["contigID"].str.split(".part").str[0]
        combo_df_w_trueclass = pd.merge(combo_df, true_clades, how = "left", left_on = "parent_genome", right_on = "genome")
        meta_data = d.split ("_")
        
        combo_df_w_trueclass["read_err"] = meta_data[1]
        combo_df_w_trueclass["model"] = "{}_{}".format(meta_data[2], meta_data[3])

        # Set label wether clade is TRUE or computed by classifier model
        if len(meta_data) >= 5:
            combo_df_w_trueclass["clade_ind"] = "{}".format(meta_data[4].lower())
        else:
            combo_df_w_trueclass["clade_ind"] = "classified"

        mean_score = combo_df_w_trueclass["pl_err"].mean()
        equal_count = (combo_df_w_trueclass["top_class"] == combo_df_w_trueclass["clade"]).sum()/len(combo_df_w_trueclass)

        mean_score_rounded = round(mean_score, 3) 
        equal_count_rounded = round(equal_count, 3) 
        print (d, mean_score_rounded, equal_count_rounded)

        # Append contig length information
        col_names = ["readID", "length", "A", "C", "G", "T", "2", "3", "4", "CpG", "tv", "ts", "CpG-ts"]
        length_df = pd.read_csv(os.path.join(length_folder, "ONT_reads_best10genomes_TOLqueries_{}_LENGTH.txt".format(meta_data[1])), sep = "\t", header = None, names = col_names)
        length_df["length_adj"] = length_df["A"] + length_df["C"] + length_df["G"] + length_df["T"]
        
        combo_df_w_trueclass_len = pd.merge(combo_df_w_trueclass, length_df, how="left", left_on = "contigID", right_on = "readID")

        # Concatenate vertically
        my_big_df = pd.concat([my_big_df, combo_df_w_trueclass_len], axis=0)
        

        """
        # Generate true classes.out output so claded_chunked_TRUE can be tested
        if "Claded" in d:
            # Select only the two columns you want
            output_df = combo_df_w_trueclass[["genome_x", "clade"]]
            output_df = output_df.rename(columns={"genome_x": "genome"})
            output_df = output_df.rename(columns={"clade": "top_class"})
            

            # Write to a tab-separated text file without the index
            output_df.to_csv("true_class_files/true_{}_classes.out".format(meta_data[1]), sep="\t", index=False)
        """

# Write to CSV file
my_big_df.to_csv("ONT_best10TOLquer_varReadErr.csv", index=False)

ONT_095_Claded_Chunked_TRUE 1.652 1.0
ONT_095_Claded_Chunked 2.995 0.936
ONT_090_Claded_Chunked_TRUE 2.06 1.0
ONT_085_Claded_Chunked_TRUE 3.306 1.0
ONT_099_Uncladed_Chunked 3.364 0.0
ONT_090_Claded_Chunked 4.307 0.904
ONT_085_Claded_Chunked 6.636 0.847
ONT_099_Claded_Chunked 2.75 0.947
ONT_099_Claded_Chunked_TRUE 1.505 1.0
ONT_1_Claded_Chunked 2.574 0.949
ONT_085_Uncladed_Chunked 8.125 0.0
ONT_095_Uncladed_Chunked 3.626 0.0
ONT_1_Uncladed_Chunked 3.31 0.0
ONT_090_Uncladed_Chunked 5.093 0.0
ONT_1_Claded_Chunked_TRUE 1.477 1.0


In [103]:
combo_df_w_trueclass_len

,contigID,count,pl_err,ratio,genome_x,top_class,parent_genome,genome_y,clade,read_err,...,G,T,2,3,4,CpG,tv,ts,CpG-ts,length_adj
0,G000008665.part_1_100,10069.0,1.0,0.0001,G000008665.part_1_100,13,G000008665,G000008665,13,1,...,1735,1538,0,0,0,672,0,0,0,6372
1,G000008665.part_1_105,10069.0,1.0,0.0001,G000008665.part_1_105,13,G000008665,G000008665,13,1,...,3648,3749,0,0,0,1344,0,0,0,15302
2,G000008665.part_1_106,10069.0,1.0,0.0001,G000008665.part_1_106,13,G000008665,G000008665,13,1,...,1929,1589,0,0,0,640,0,0,0,6784
3,G000008665.part_1_107,10069.0,1.0,0.0001,G000008665.part_1_107,13,G000008665,G000008665,13,1,...,1963,1848,0,0,0,692,0,0,0,7491
4,G000008665.part_1_108,10069.0,3.0,0.0003,G000008665.part_1_108,13,G000008665,G000008665,13,1,...,2472,2488,0,0,0,1290,0,0,0,10176
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1864,G900089075.part_9_95,10069.0,0.0,0.0000,G900089075.part_9_95,6,G900089075,G900089075,6,1,...,1010,1590,0,0,0,490,0,0,0,5077
1865,G900089075.part_9_96,10069.0,0.0,0.0000,G900089075.part_9_96,6,G900089075,G900089075,6,1,...,1339,2558,0,0,0,684,0,0,0,7708
1866,G900089075.part_9_97,10069.0,3.0,0.0003,G900089075.part_9_97,6,G900089075,G900089075,6,1,...,3562,7311,0,0,0,1498,0,0,0,21192
1867,G900089075.part_9_99,10069.0,0.0,0.0000,G900089075.part_9_99,6,G900089075,G900089075,6,1,...,4900,7860,0,0,0,2200,0,0,0,26412


In [84]:
len(length_df)

1860

In [ ]:
# Generate true classes for 

In [58]:
# Merge dataframes

total_p1 = "/Users/nora/Documents/ml_metagenomics/tol_variable_k_resuls/k7_v38_8k_s28_clade_All_TOL_CONTIGS_Chunked_Model"
total_p2 = "/Users/nora/Documents/ml_metagenomics/tol_variable_k_resuls/k7_v38_8k_s28_clade_All_TOL_CONTIGS_Full_Model"
total_p3 = "/Users/nora/Documents/ml_metagenomics/tol_variable_k_resuls/k7_v38_8k_s28_clade_All_TOL_CONTIGS_Global_Model"
total_p4 = "/Users/nora/Documents/ml_metagenomics/tol_variable_k_resuls/k7_v38_8k_s28_clade_All_TOL_CONTIGS_ChunkedExp_Model"
total_p5 = "/Users/nora/Documents/ml_metagenomics/tol_variable_k_resuls/k7_v38_8k_s28_clade_All_TOL_CONTIGS_ChunkedExp_Global_hybrid_Model"
total_p6 = "/Users/nora/Documents/ml_metagenomics/tol_variable_k_resuls/k7_v38_8k_s28_clade_All_TOL_CONTIGS_ChunkedExp_Global_hybrid_Model"
total_p7 = "/Users/nora/Documents/ml_metagenomics/tol_variable_k_resuls/k7_v38_8k_s28_clade_All_TOL_CONTIGS_Global_Chunk_Model"
total_p8 = "/Users/nora/Documents/ml_metagenomics/tol_variable_k_resuls/k7_v55_8k_s28_clade_All_TOL_CONTIGS_ChunkedExp_Model_v55"
total_p9 = "/Users/nora/Documents/ml_metagenomics/tol_variable_k_resuls/k7_v38_8k_s28_clade_All_TOL_CONTIGS_ChunkedExp_Model_TRUE_CLADE"
total_p10 = "/Users/nora/Documents/ml_metagenomics/tol_variable_k_resuls/k7_v38_8k_s28_clade_All_TOL_CONTIGS_Full_Model_TRUE_CLADE"

total_f1 = "summary_FULL_GENOME_CONTIGS.pl_error"
total_f2 = "summary_FULL_GENOME_CONTIGS_FullModel.pl_error"
total_f3 = "summary_FULL_GENOME_CONTIGS_GlobalModel.pl_error"
total_f4 = "summary_FULL_GENOME_CONTIGS_ChunkExpModel.pl_error"
total_f5 = "summary_FULL_GENOME_CONTIGS_ChunkExp_Global_Model.pl_error"
total_f6 = "summary_FULL_GENOME_CONTIGS_ChunkExp_Global_Modelp_0.9.pl_error"
total_f7 = "summary_FULL_GENOME_CONTIGS_GlobalChunkModel.pl_error"
total_f8 = "summary_FULL_GENOME_CONTIGS_ChunkExpModel_v55.pl_error" # incomplete classification
total_f9 = "summary_FULL_GENOME_CONTIGS_ChunkExpModel_TRUE_CLADE.pl_error"
total_f10 = "summary_FULL_GENOME_CONTIGS_FullModel_TRUE_CLADE.pl_error"



t1 = pd.read_csv(os.path.join(total_p1, total_f1), sep=' ', header = 0)
t2 = pd.read_csv(os.path.join(total_p2, total_f2), sep=' ', header = 0)
t3 = pd.read_csv(os.path.join(total_p3, total_f3), sep=' ', header = 0)
t4 = pd.read_csv(os.path.join(total_p4, total_f4), sep=' ', header = 0)
t5 = pd.read_csv(os.path.join(total_p5, total_f5), sep=' ', header = 0)
t6 = pd.read_csv(os.path.join(total_p6, total_f6), sep=' ', header = 0)
t7 = pd.read_csv(os.path.join(total_p7, total_f7), sep=' ', header = 0)
t8 = pd.read_csv(os.path.join(total_p8, total_f8), sep=' ', header = 0)
t9 = pd.read_csv(os.path.join(total_p9, total_f9), sep=' ', header = 0)
t10 = pd.read_csv(os.path.join(total_p10, total_f10), sep=' ', header = 0)


In [59]:
t1["model"] = "Chunk"
t2["model"] = "Full"
t3["model"] = "Global"
t4["model"] = "ChunkExp"
t7["model"] = "Global_Chunk"
t8["model"] = "ChunkExp_v55"
t9["model"] = "ChunkExp_TrueClade"
t10["model"] = "Full_TrueClade"

In [60]:
total_t = pd.concat([t1, t2, t3, t4, t5, t6, t7, t8, t9, t10], ignore_index=True)

In [6]:
total_t.to_csv(os.path.join(os.getcwd(), "summary_TOL_full_genome_contigs_six_models.pl_error"), index=False, sep=' ', header=True)